In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time
import os
from PIL import Image
from io import BytesIO

# Set up the webdriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open the Freepik search page for apple images
url = 'https://yandex.com/images/search?lr=10616&text=apple+fruit&itype=jpg'
driver.get(url)

# Give the page time to load
time.sleep(10)

# Function to check if the image is relevant and clean
def is_valid_image(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            # Check if the image is too small
            if img.size[0] > 100 and img.size[1] > 100:
                return True
    except:
        return False
    return False

# Scroll and load more images until we have enough
image_urls = set()
while len(image_urls) < 300:
    # Scroll to the bottom of the page
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(20)  # Give time to load more images
    
    # Find image elements and extract URLs
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'img')
    new_image_urls = {img.get_attribute('src') for img in image_elements if img.get_attribute('src')}
    
    # Filter and add only valid image URLs
    for url in new_image_urls:
        if is_valid_image(url):
            image_urls.add(url)
    
    print(f"Found {len(image_urls)} unique relevant image URLs so far.")

# Create a directory to save images
if not os.path.exists('apple_images'):
    os.makedirs('apple_images')

# Download and save images
for i, url in enumerate(image_urls):
    if i >= 300:
        break
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(f'apple_images/apple_image_{i+1}.png', 'wb') as file:
                file.write(response.content)
    except Exception as e:
        print(f"Could not download {url}. Error: {e}")

# Close the driver
driver.quit()


Found 63 unique relevant image URLs so far.
Found 63 unique relevant image URLs so far.
Found 63 unique relevant image URLs so far.
Found 63 unique relevant image URLs so far.
Found 93 unique relevant image URLs so far.
Found 361 unique relevant image URLs so far.
